# RAG System using LlamaIndex and LanceDB

This notebook demonstrates how to build a Retrieval-Augmented Generation (RAG) system for question answering on the jp-handbook-full.pdf document using:
- LlamaIndex for document processing and querying
- LanceDB as the vector store

## Install Required Dependencies

First, let's install the necessary libraries if they aren't already installed.

In [4]:
%%capture
# Install required packages
! pip install llama-index llama-index-readers-file llama-index-llms-google-genai llama-index-embeddings-google-genai lancedb pypdf python-dotenv google-generativeai

## Import Libraries

Now let's import the necessary libraries for our RAG system.

In [5]:
# Import required libraries
import os
import lancedb
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader  # Updated import path
from llama_index.core import Settings, VectorStoreIndex, StorageContext  # Core imports
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter  # Updated import path

# Load environment variables from .env file
load_dotenv()

# Set up the Gemini models for LLM and embeddings
import google.generativeai as genai

# Get API key from environment variables
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    print("Warning: GOOGLE_API_KEY not found in environment variables.")
    print("Please create a .env file in the project root with your Google API key:")
    print("GOOGLE_API_KEY=your-api-key-here")
else:
    print("Google API key loaded successfully from environment variables.")
    genai.configure(api_key=api_key)

    # Set up the LLM using gemini-2.0-flash
    llm = GoogleGenAI(model="gemini-2.0-flash", temperature=0.1)

    # Set up the embedding model using gemini-embedding-exp
    embed_model = GoogleGenAIEmbedding(model_name="models/text-embedding-004", api_key=api_key)

    # Configure LlamaIndex to use these models
    Settings.llm = llm
    Settings.embed_model = embed_model

/Users/josereyes/Dev/poc-modal/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google API key loaded successfully from environment variables.


## Load and Process the PDF Document

We'll load the jp-handbook-full.pdf document and split it into chunks for better retrieval.

In [6]:
# Define the path to the PDF document
pdf_path = '../jp-handbook-full.pdf'

# Check if the file exists
if os.path.exists(pdf_path):
    print(f"Loading document: {pdf_path}")
else:
    print(f"Error: Document not found at {pdf_path}")

# Load the document using SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=[pdf_path]).load_data()
print(f"Loaded {len(documents)} document(s)")

Loading document: ../jp-handbook-full.pdf
Loaded 132 document(s)
Loaded 132 document(s)


## Parse the Document into Nodes

Split the document into smaller chunks for more effective retrieval.

In [7]:
# Create a node parser to split the document into chunks
node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)

# Parse the document into nodes
nodes = node_parser.get_nodes_from_documents(documents)
print(f"Document split into {len(nodes)} nodes")

Document split into 132 nodes


## Set Up LanceDB Vector Store

Create a LanceDB vector store to efficiently store and retrieve document vectors.

In [10]:
# Create LanceDB database and table
db = lancedb.connect('./lancedb')
table_name = 'jp_handbook'

# Remove existing table if it exists
if table_name in db.table_names():
    db.drop_table(table_name)

# Create a new LanceDB vector store
vector_store = LanceDBVectorStore(db=db, table_name=table_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create an index from the nodes
index = VectorStoreIndex(nodes, storage_context=storage_context)
print("Vector index created successfully")

Vector index created successfully


## Create a Query Engine

Set up a query engine to answer questions based on the indexed document.

In [11]:
# Create a query engine
query_engine = index.as_query_engine(similarity_top_k=5)
print("Query engine ready for questions")

Query engine ready for questions


## Ask Questions

Now you can ask questions about the jp-handbook-full.pdf document.

In [23]:
def ask_question(question, num=0):
    # print(f"Q: {question}")
    try:
        response = query_engine.query(question)
        markdown_response = f"""
### Question {f"#{num}" if num != 0 else ""}
{question}

### Answer
{response}

### Sources
"""
        for i, source_node in enumerate(response.source_nodes):
            content_preview = source_node.node.get_content()[:150].replace('\n', ' ').strip()
            markdown_response += f"{i+1}. {content_preview}...\n\n"
            
        from IPython.display import Markdown, display
        display(Markdown(markdown_response))
        return response
    except Exception as e:
        error_message = f"""
### Question
{question}

### Error
```
{str(e)}
```

If this is a RESOURCE_EXHAUSTED error, you've hit Google API quota limits. Try again later.
"""
        from IPython.display import Markdown, display
        display(Markdown(error_message))
        return None

In [13]:
# Example questions - replace with your own questions about the handbook
question = "List the duties and services of a Justice of the Peace."
response = ask_question(question)


### Question
List the duties and services of a Justice of the Peace.

### Answer
A JP can witness statutory declarations and affidavits, certify copies of original documents, and perform other functions given to them by or under a NSW Act or NSW Regulation. These other functions may include witnessing various kinds of legal documents and administering special kinds of oaths.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. 3Introduction  1  Introduction  Thank you for serving the people of NSW as a JP.  The Attorney General of NSW acknowledges and  sincerely appreciates...

4. 11Performing your functions as a Justice of the Peace  2  Performing your functions   as a Justice of the Peace  This section details the most commonl...

5. 2.4  Performing other functions of a JP  What is it? As a JP, you can only per form functions that are given to  a JP by or under a NSW Act or NSW Reg...



In [14]:
# Try another question
question = "List the procedures for arresting a criminal?"
response = ask_question(question)


### Question
List the procedures for arresting a criminal?

### Answer
I am sorry, but the information needed to answer this question is not available in the context.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. Justice of the Peace Handbook 122...

4. ...

5. ...



## Multiple Q&A



In [26]:
import time

# List of questions to ask about the JP handbook
jp_questions = [
    "What are the main responsibilities of a Justice of the Peace?",
    "What legal documents can a Justice of the Peace certify or witness?",
    "What is the procedure for handling bail applications?",
    "What ethical considerations should a Justice of the Peace keep in mind?",
    "How does a Justice of the Peace handle conflicts of interest?",
    "What are the requirements to become a Justice of the Peace?",
    "What are the limitations on a Justice of the Peace's authority?",
    "How should a Justice of the Peace handle confidentiality issues?",
    "What training is required for a Justice of the Peace?",
    "What record-keeping responsibilities does a Justice of the Peace have?"
]

# Function to iterate through questions with a delay to avoid rate limiting
def ask_multiple_questions(questions_list, delay_seconds=1):
    print(f"Processing {len(questions_list)} questions with {delay_seconds} second delay between each...\n")
    for i, question in enumerate(questions_list):
        ask_question(question, i+1)
        # print(f"\nWaiting {delay_seconds} seconds before next question...")
        time.sleep(delay_seconds)

In [27]:
# Call the function to process all questions
# You can set a longer delay if you encounter rate limiting issues
response = ask_multiple_questions(jp_questions)


Processing 10 questions with 1 second delay between each...




### Question #1
What are the main responsibilities of a Justice of the Peace?

### Answer
The handbook includes information about the functions, obligations, and available support for Justices of the Peace. It also contains frequently asked questions about their authority and a code of conduct.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. 3Introduction  1  Introduction  Thank you for serving the people of NSW as a JP.  The Attorney General of NSW acknowledges and  sincerely appreciates...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. Justice of the Peace Handbook 122...




### Question #2
What legal documents can a Justice of the Peace certify or witness?

### Answer
A Justice of the Peace can witness a statutory declaration or an affidavit, and they can certify copies of original documents.


### Sources
1. Justice  of the Peace  Handbook...

2. 11Performing your functions as a Justice of the Peace  2  Performing your functions   as a Justice of the Peace  This section details the most commonl...

3. 128 Justice of the Peace Handbook...

4. QUICK REFERENCE:  Certifying a copy of an original  document  STEP 1 Have the original document and copy in front of you  STEP 2 Che ck that the copy...

5. Justice of the Peace Handbook 122...




### Question #3
What is the procedure for handling bail applications?

### Answer
The document does not contain information about the procedure for handling bail applications.


### Sources
1. Justice  of the Peace  Handbook...

2. Justice of the Peace Handbook 122...

3. 128 Justice of the Peace Handbook...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. ...




### Question #4
What ethical considerations should a Justice of the Peace keep in mind?

### Answer
A Justice of the Peace should not unreasonably refuse services, but rather treat all persons with courtesy, dignity, and respect, and in a timely manner. They must not engage in dishonest activities or conduct themselves in a way that brings their office into disrepute. They are expected to keep private information safe and remain independent and impartial, declining service if a conflict of interest exists. They cannot charge a fee or accept gifts for providing services and must not use their title to advance personal interests, although they may use the title on business cards or letterheads. If their term expires or they are removed from office, they must immediately cease providing services.


### Sources
1. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

2. Justice  of the Peace  Handbook...

3. 1 Access to services  1) A Justice of the Peace must not unreasonably refuse to provide  Justice of the Peace services and must treat all persons seek...

4. 128 Justice of the Peace Handbook...

5. Justice of the Peace Handbook 122...



...


### Question #6
What are the requirements to become a Justice of the Peace?

### Answer
The context does not provide information about the requirements to become a Justice of the Peace.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. Justice of the Peace Handbook 122...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. 1 Access to services  1) A Justice of the Peace must not unreasonably refuse to provide  Justice of the Peace services and must treat all persons seek...




### Question #7
What are the limitations on a Justice of the Peace's authority?

### Answer
A Justice of the Peace must not refuse to provide services without a reasonable cause and must treat everyone with courtesy, dignity, and respect. They must act in a timely manner. They are prohibited from engaging in dishonest activities or behaving in a way that discredits the position. They are required to protect private, confidential, or commercially sensitive information obtained while providing services, unless authorized by law. They must remain independent and impartial, and decline service if a conflict of interest exists. If not reappointed or removed from office, they must immediately stop providing services. They cannot charge a fee or accept gifts for providing services. While they can use the title on business cards or letterheads, they cannot use the title to advance personal, commercial, or business interests.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. Justice of the Peace Handbook 122...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. 1 Access to services  1) A Justice of the Peace must not unreasonably refuse to provide  Justice of the Peace services and must treat all persons seek...




### Question #8
How should a Justice of the Peace handle confidentiality issues?

### Answer
A Justice of the Peace must protect private, confidential, or commercially sensitive information obtained while providing services, unless authorized by law to reveal it.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

4. Justice of the Peace Handbook 122...

5. 1 Access to services  1) A Justice of the Peace must not unreasonably refuse to provide  Justice of the Peace services and must treat all persons seek...




### Question #9
What training is required for a Justice of the Peace?

### Answer
There is no mention of required training in the provided context.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. Justice of the Peace Handbook 122...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. 3Introduction  1  Introduction  Thank you for serving the people of NSW as a JP.  The Attorney General of NSW acknowledges and  sincerely appreciates...




### Question #10
What record-keeping responsibilities does a Justice of the Peace have?

### Answer
A Justice of the Peace is required to inform the Department of Justice in writing about any changes to their name, postal or email address, or the telephone number they use for Justice of the Peace services as soon as it is practical.


### Sources
1. Justice  of the Peace  Handbook...

2. 128 Justice of the Peace Handbook...

3. Justice of the Peace Handbook 122...

4. 123 Code of Conduct for Justices of the Peace  5  Code of Conduct for   Justices of the Peace...

5. 2)  A Justice of the Peace must notify the Department of Justice in  writing of any of the following changes as soon as practicable   after that chang...



## Interactive Q&A

You can use the cell below to interactively ask questions about the handbook.

In [28]:
# Interactive question answering
question = input("Ask a question about the handbook:")
response = ask_question(question)


### Question 
what is a statutory declaration?

### Answer
A statutory declaration will contain the words ‘statutory declaration’ and the name of the relevant state, territory or Commonwealth law. A NSW statutory declaration is made under the Oaths Act 1900 (NSW) and can be in the format of the Eighth Schedule and the Ninth Schedule of the Act. It may be either hand-written or type-written, but must be written in English.


### Sources
1. STEP 11 Follow the fnal steps for the declarant’s declaration  STEP 12 F ollow the fnal steps for the declarant’s declaration  • wa tch the declarant...

2. Step-by-step  procedure   STEP 1  Check if the document is a statutory declaration  You can identify a statutory declaration because it will contain:...

3. Justice  of the Peace  Handbook...

4. QUICK REFERENCE:  Witnessing a statutory declaration  STEP 1 Check the document is a statutory declaration  STEP 2 S ee the client’s face  STEP 3 Conf...

5. Commonwealth statutory declaration  An editable template can be downloaded from the JP website  www.jp.nsw.gov.au  Commonwealth of Australia  STATUTOR...



## Conclusion

This RAG system allows you to ask questions about the jp-handbook-full.pdf document and get contextually relevant answers based on the content. Feel free to modify the query parameters or add more advanced features as needed.